# Generate Counterfactuals by Distance

In [ ]:
import numpy as np
from src.datasets import IMDBDataset
from src.models import AnalysisModels
from src.analyzers.svm import SVMDistanceAnalyzer

In [ ]:
ds = IMDBDataset(config_path="./configs/datasets/imdb.yaml", root="datasets/imdb")
models = AnalysisModels(config_path="./configs/models/analysis-models.yaml", root="models/analysis-models")
model = models.svm.model
analyzer = SVMDistanceAnalyzer(model, ds, "datasets/imdb/svm_buffer.json")

In [ ]:
import json
print(json.dumps(analyzer.get_counterfactual_examples([
    "I would like to remind that this movie was advertised as a real-life story. But what is this?. A waste of my good money!",
    "This is the best movie I had watched so far. The marvelous CGI and super story line successfully kept the eyes of the audience fixed."
]), indent=4))

# Generate Counterfactuals by Opposite Neighbourhood

## SVM Theory
What SVM does
$$
\boxed{"prompt"}\rightarrow\boxed{Vector_{TFIDF}\ (i.e., x)}\rightarrow\boxed{Vector_{SVM}\ (i.e., \phi(x))}\\
x\in R^m,\ \phi(x)\in R^n.
$$
Note that here $m$ is the number of dimensions in the vector space of the TFIDF Vectorizer and $n$ is the number of dimensions in the vector space learnt by the SVM. $\phi(x)$ is known as the kernel function. Hence, the vector space learnt by the SVM is commonly known as the **output vector space of the kernel function**.

Once the SVM learns the vector space of $\phi(x)$, it finds the best hyperplane that satisfies $w^T.\phi(x)+b=0$. Note that $w$ are the coefficients with size $n$. This equation can be expanded as $w_1\phi_1(x)+w_2\phi_2(x)+w_3\phi_3(x)...+w_n\phi_n(x)+b$

## Our method
We will be using the following method to generate counter factuals for a given $prompt$.
1. Generate a set of contradictory prompts for the given prompt.
2. Vectorize all the prompts using the TFIDF vetorizer into the vector space $X$.
3. Project all the vectors into the SVM's kernel space $K$.
4. Find the mirror point of the given prompt's TFIDF vector on the hyperplane of the SVM ($C$).
5. Out of the vectors of the contradictory prompts, find the closest point to $C$. Then the prompt corresponding to this point will be returned

### 1. Contradictory prompt generation

We will generate contradictory prompts for a given prompt $prompt_0$ using a finetuned T5 model/ custom WordFlippingGenerator. These new prompts will be $[contradictory\_prompt_i]$


### 2. Vectorize into $X$
$$
\boxed{"prompt_0"}\rightarrow\boxed{Vector_{TFIDF}\ (i.e., x_0)}
$$
The $prompt_0$ will be mapped to $x_0$ from the TFIFT vectorizer. The $[contradictory\_prompt_i]$ s will be mapped to $[x_{c,i}]$

### 3. Project into $K$
SVM is already learnt. i.e., we know the kernel function ($\phi(.)$), coefficients ($w$), and the bias ($b$). Hence, we will project $x_0$ and $[x_{c,i}]$ into $K$

$$
\boxed{Vector_{TFIDF}\ (i.e., x_0)}\rightarrow\boxed{Vector_{SVM}\ (i.e., \phi(x_0)\in K)}
$$
We will call this $\phi(x_0)$ as $A$ for simplicity

##### $\phi(.)$ when the kernel is RBF
Assume that a single TFIDF vector will have the size $n$ and the number of support vectors will be $m$. The RBF kernel is given by
$$
K( \overrightarrow{x}, \overrightarrow{l^m})=e^{-\gamma{||\overrightarrow{x}-\overrightarrow{l^m}||}^2}
$$
Here $x$ is a vector in the TFIDF vector space with size $m$. $l^m$ is a collection of $m$ vectors (i.e., the support vectors) with each of size $n$.

### 4. Find $A$'s mirror point ($C$)
Once we have $\phi(x_0)$ for the given prompt, we find its opposite projection on the hyperplane of the SVM characterised by $w$ and $b$. For simplicity, we'll call $\phi(x_0)$ as $A$ and $hyperplane$ as $h$.
$$
hyperplane=h\equiv(w_1, w_2,...w_n, b) \\
\phi(x_0)=A=(a_1,a_2,...a_n)
$$
Any line $l$ which is normal to the $h$ through $A$ will be given by the parametric equation
$$
l\equiv A+tw=0\\
l\equiv (a_1+tw_1, a_2+tw_2,...,a_n+tw_n)
$$
Let $t$ take the value $t_0$ at the point $B$ that lies on this line and the hyperplane.
$$
B\equiv (a_1+t_0w_1, a_2+t_0w_2,...,a_n+t_0w_n)
$$
Since this point would also satisfy the hyperplane,
$$
w^T.l_0+b=0 \\
(w_1(a_1+t_0w_1), w_2(a_2+t_0w_2),...,w_n(a_n+t_0w_n))+b=0\\
t_0=-\frac{(b+w^T.A)}{||w||_2^2}
$$
The mirror point $C$ will exist where $t=2t_0$. Hence,
$$
C\equiv (a_1+2t_0w_1, a_2+2t_0w_2,...,a_n+2t_0w_n)
$$

##### Example

Reflection of point $A(3,1,2)$ on the hyperplane $x+2y+z=1$ (Note that here, $(3,1,2)=(a_1,a_2,a_3)$, $(1,2,1)=(w_1,w_2,w_3)$, and $-1=b$)
1. Construct the line normal to the plane that intersects point $A(3,1,2)$:
  $$
  line (x,y,z)=(3,1,2)+t(1,2,1)
  $$
  (Any line normal to the plane $x+2y+z=c$ will move in the direction $(1,2,1)$)

2. Find the point B on the normal line that intersects the plane:
  $$
  (3+t)+2(1+2t)+(2+t)=1
  $$

  Solving, we get $t=-1$. Hence the intersection point $B$ (on the plane) is at $(2,-1,1)$.

3. Point $A$ is at $t=0$, and point $B$ is at $t=-1$, so the mirror image of $A$, say $\hat{A}$ will be twice the distance, at $t=-2$:
  $$
  \boxed{ \hat{A} \equiv (1,-3,0)}
  $$

### 5. Find the closest point to C and retreive the contradictory prompt
Now that the mirror point and contradictory points are all in the kerrnel space, the distance to the contradictory points from the point $C$ will be found. The prompt will be selected as the one which yields the smallest distance to the point $C$.


## Tests

In [ ]:
import numpy as np
from src.models import AnalysisModels
from src.datasets import IMDBDataset
from tqdm.auto import tqdm

models = AnalysisModels("./configs/models/analysis-models.yaml", "./models/analysis-models")
dataset = IMDBDataset("./configs/datasets/imdb.yaml", "./datasets/imdb/")

svc_rbf = models.svm.model
x = dataset.x_test.toarray()
p = np.random.randn(x.shape[1])

### Linear kernel

In [ ]:
import numpy as np

# Your code here (with the provided function)
from sklearn.svm import SVC
import numpy as np

X_train = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
y_train = np.array([0, 0, 1, 1])

svm = SVC(kernel='linear', degree=3)
svm.fit(X_train, y_train)

def get_mirror_point_linear(svm, qp):
    w = svm.coef_[0]
    b = svm.intercept_[0]
    t = -(b+w.dot(qp))/(np.linalg.norm(w)**2)
    mp = qp + 2*t*w
    return mp

query_point = np.array([13, 5])
mirror_point = get_mirror_point_linear(svm, query_point)

print(f"Query point: {query_point}, Distance: {svm.decision_function([query_point])}")
print(f"Mirror point: {mirror_point}, Distance: {svm.decision_function([mirror_point])}")


### Polynomial Kernel

In [ ]:
import numpy as np

# Your code here (with the provided function)
from sklearn.svm import SVC
import numpy as np

X_train = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
y_train = np.array([0, 0, 1, 1])

# Linear kernel SVM
svm_linear = SVC(kernel='linear', degree=3)
svm_linear.fit(X_train, y_train)

# Polynomial kernel SVM
svm_poly = SVC(kernel='poly', degree=3)
svm_poly.fit(X_train, y_train)

def get_k_poly(svm, x):
    svs = svm.support_vectors_
    d = svm.degree
    gamma = svm._gamma
    print(svs.shape, x.shape)
    k = (gamma * np.dot(svs, x.T) + 1) ** d
    return k

def distance_to_hyperplane(svm_model, point):
    # Get the support vectors, coefficients, and bias from the trained SVM model
    support_vectors = svm_model.support_vectors_
    dual_coefficients = svm_model.dual_coef_[0]
    bias = svm_model.intercept_

    # Get the kernel coefficient and degree from the trained SVM model
    gamma = svm_model._gamma
    degree = svm_model.degree

    # Compute the distance to the hyperplane
    distance = 0.0
    for i in range(len(support_vectors)):
        # Calculate the polynomial kernel between the support vector and the given point
        kernel_value = (gamma * np.dot(support_vectors[i], point) + 1) ** degree

        # Update the distance using the support vector and kernel value
        distance += dual_coefficients[i] * kernel_value

    # Add the bias term to the distance
    distance += bias

    return distance

p = np.array([
    [-8,2],
    [-3,3],
    [-2,4],
    [-1,5]
])
# svm_poly.decision_function(p), distance_to_hyperplane(svm_poly, p[0])
get_k_poly(svm_poly, p).shape


### RBF Kernel Analysis

#### Implementation

##### Method 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_circles
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = make_circles(n_samples=500, noise=0.06, random_state=42)

df = pd.DataFrame(dict(x1=X[:, 0], x2=X[:, 1], y=y))

In [ ]:
colors = {0:'blue', 1:'yellow'}
fig, ax = plt.subplots()
grouped = df.groupby('y')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x1', y='x2', label=key, color = colors[key])
plt.show()

In [ ]:
def RBF(X, gamma):

    # Free parameter gamma
    if gamma == None:
        gamma = 1.0/X.shape[1]

    # RBF kernel Equation
    K = np.exp(-gamma * np.sum((X - X[:,np.newaxis])**2, axis = -1))

    return K

In [ ]:
clf_rbf = SVC(kernel="rbf")
clf_rbf.fit(X, y)
K = RBF(X, gamma=clf_rbf._gamma)

In [ ]:
clf = SVC(kernel="linear")

clf.fit(K, y)

pred = clf.predict(K)

print("Accuracy: ",accuracy_score(pred, y))

##### Method 2

In [ ]:
import numpy as np

from sklearn.svm import SVC
import numpy as np

X_train = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
y_train = np.array([0, 0, 1, 1])

svm = SVC(kernel='rbf', gamma='scale', degree=3)
svm.fit(X_train, y_train)

def get_distance_rbf(svm, qp):
    # Calculate the distance from the query point to each support vector
    sv = svm.support_vectors_
    distances = np.linalg.norm(sv - qp, axis=1)

    # Use the decision function to get the weight (distance) for each support vector
    decision_values = svm.decision_function([qp])[0]
    weights = np.exp(-svm._gamma * (distances ** 2)) * decision_values

    # Calculate the weighted average of the support vectors to obtain the approximate mirror point
    weighted_average = np.average(sv, axis=0, weights=weights)

    return weighted_average

query_point = np.array([1, 5])
mirror_point = get_distance_rbf(svm, query_point)

print(f"Query point: {query_point}, Distance: {svm.decision_function([query_point])}")
print(f"Mirror point: {mirror_point}, Distance: {svm.decision_function([mirror_point])}")


##### Method 3

In [ ]:
import numpy as np
from sklearn.svm import SVC

def rbf_kernel(x, y, gamma):
    return np.exp(-gamma * np.linalg.norm(x - y) ** 2)

def distance_to_hyperplane(clf, x_i):
    # Get support vectors and dual coefficients
    support_vectors = clf.support_vectors_
    dual_coefficients = clf.dual_coef_.ravel()
    gamma = clf._gamma

    # Compute the decision function value
    decision_function_value = 0
    for i in range(len(support_vectors)):
        decision_function_value += dual_coefficients[i] * rbf_kernel(support_vectors[i], x_i, gamma)

    decision_function_value += clf.intercept_

    # Compute the distance
    norm_w = np.linalg.norm(clf.dual_coef_ @ support_vectors)
    distance = decision_function_value / norm_w
    return distance

X_train = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
y_train = np.array([0, 0, 1, 1])
x_i = [-1,1]

gamma = 0.1
clf = SVC(kernel='rbf', gamma=gamma)
clf.fit(X_train, y_train)

distance_to_x_i = distance_to_hyperplane(clf, x_i)
distance_to_x_i, clf.decision_function([x_i])

### RBF Kernel

In [ ]:
def split_matrix(mat, max_sz):
    num_rows = mat.shape[0]
    num_splits = (num_rows - 1) // max_sz + 1
    split_matrices = []

    for i in range(num_splits):
        start_idx = i * max_sz
        end_idx = min((i + 1) * max_sz, num_rows)
        split_matrices.append(mat[start_idx:end_idx])

    return tuple(split_matrices)

def calc_dif_norms(l_mat, s_mat, axis=1, max_sz=1000, show_prog=False):
    mat_tup = split_matrix(l_mat, max_sz)
    norms = []

    if show_prog: print("Calculating norms...")
    for m in tqdm(mat_tup, disable=not show_prog):
        n_m = np.expand_dims(m, axis=1)
        norm_batch = np.linalg.norm(n_m-s_mat, axis=1)
        norms.extend(norm_batch)
    norms = np.array(norms)
    return norms

def rbf(x, model):
    gamma = model._gamma
    svs = model.support_vectors_.toarray()
    norms = calc_dif_norms(svs, x, show_prog=True)
    k = np.exp(-gamma*norms)
    return k


k = rbf(x[:2], svc_rbf)
k.shape

## Final Implementation

### Counterfactual Generator: T5

In [ ]:
from src.analyzers import SVMAnalyzer
analyzer = SVMAnalyzer(
    svm_path="./models/analysis-models/svm.pkl",
    vectorizer_path="./models/analysis-models/tfidf.pkl",
    cf_generator_config_path="./configs/models/t5-cf-generator.yaml",
    cf_generator_root="./models/cf-generator"
)

In [ ]:
review = "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."
search_space = 2
cf = analyzer(review, search_space)

In [ ]:
explanation = analyzer.explanation()
print(explanation)

### Counterfactual Generator: WordFlipping

#### Predefined configuration

In [ ]:
from src.analyzers import SVMAnalyzer
analyzer = SVMAnalyzer(
    svm_path="./models/analysis-models/svm.pkl",
    vectorizer_path="./models/analysis-models/tfidf.pkl",
    cf_generator_config="./configs/models/wf-cf-generator.yaml"
)
review = "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."
search_space = 2
cf = analyzer(review, search_space)
explanation = analyzer.explanation()
print(explanation)

#### Test bench

In [ ]:
from src.test_bench import TestBench

configurations = [
    {
        "name": "adjectives",
        "generator_config": {
            "sample_prob_decay_factor": 0.2,
            "flip_prob": 0.5,
            "flipping_tags": ["JJ", "JJR", "JJS"],
        },
    },
    {
        "name": "nouns",
        "generator_config": {
            "sample_prob_decay_factor": 0.2,
            "flip_prob": 0.5,
            "flipping_tags": ["NN", "NNP", "NNPS", "NNS"],
        },
    },
    {
        "name": "adverbs",
        "generator_config": {
            "sample_prob_decay_factor": 0.2,
            "flip_prob": 0.5,
            "flipping_tags": ["RB", "RBR", "RBS", "RP"],
        },
    },
    {
        "name": "verbs",
        "generator_config": {
            "sample_prob_decay_factor": 0.2,
            "flip_prob": 0.5,
            "flipping_tags": ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
        },
    },
]
text="One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."

tb = TestBench(
    model_path="./models/analysis-models/svm.pkl",
    vectorizer_path="./models/analysis-models/tfidf.pkl",
    analyzer_name="svm",
    cf_generator_config="./configs/models/wf-cf-generator.yaml",
)

In [ ]:
reports = tb(configurations, text, 2)

In [ ]:
for report in reports:
    print(report)
    print()

In [ ]:
from src.datasets import IMDBDataset

ds = IMDBDataset(config_path="./configs/datasets/imdb.yaml", root="datasets/imdb")
tb.evaluate(ds.x_test, ds.y_test, save_dir="evaluations/svm")